# Last.fm

In [10]:
import requests
import json
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from random import sample

### Function used for data scripting

In [3]:
#shared secret:ae1ca19ada9ac3d11d2a2127b339545b
def lastfm_get(payload):

    headers = {'user-agent': 'rzhan97'}
    url = 'http://ws.audioscrobbler.com/2.0/'

    payload['api_key'] = '5e11e4bca8ad54566e466513df709822'
    payload['format'] = 'json'

    response = requests.get(url, headers=headers, params=payload)
    return response

#This function is only help for me to understand json file
def jprint(obj):
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

In [4]:
def lookup_userinfo(user):
    response = lastfm_get({
    'method': 'user.getInfo',
    'user' : user
    })

    if response.status_code != 200:
        return None

    name = response.json()['user']['realname']
    country = response.json()['user']['country']
    playcount = response.json()['user']['playcount']


    # rate limiting
    if not getattr(response, 'from_cache', False):
        time.sleep(0.25)
    return name,country,playcount

In [5]:
#Build a funtion to get user's top track, I also wrote a function about user's loved song and recent songs
#However, recent songs cannot represent user like it or not, and the loved song 
def lookup_topsong(user):
    response = lastfm_get({
    'method': 'user.getTopTracks',
    'user' : user
    })

    if response.status_code != 200:
        return None

    song = [t['name'] for t in response.json()['toptracks']['track']]
    playcount = [t['playcount'] for t in response.json()['toptracks']['track']]
    artist = [t['artist']['name'] for t in response.json()['toptracks']['track']]


    # rate limiting
    if not getattr(response, 'from_cache', False):
        time.sleep(0.25)
    return song,playcount,artist

In [6]:
#Build a funtion to get user's library
def lookup_songinfo(track,artist):
    response = lastfm_get({
    'method': 'track.getInfo',
    'track' : track,
    'artist':artist
    })

    if response.status_code != 200:
        return None

    duration = response.json()['track']['duration']
    listeners = response.json()['track']['listeners']
    playcount = response.json()['track']['playcount']
    album = response.json()['track']['album']['title'] 
    tags = [t['name'] for t in response.json()['track']['toptags']['tag'] ]
    try:
        published_date = response.json()['track']['wiki']['published']
    except:
        published_date = None

    # rate limiting
    if not getattr(response, 'from_cache', False):
        time.sleep(0.25)
    return duration,listeners,playcount,album,tags,published_date

In [1]:
#Get my neighbors' neighbors' neighbors, which is 3rd connection with me, to reduce some bias
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import time
def get_users(username):
    path = "/Users/zoe/Desktop/chromedriver"
    driver = webdriver.Chrome(path)
    driver.get("https://www.last.fm/user/"+username+"/neighbours")
    user = []
    user_names = driver.find_elements_by_class_name("user-list-name")
    for i in range(len(user_names)):
        while True:
            try:
                time.sleep(2)
                user_link = user_names[i].find_element_by_tag_name('a').get_attribute('href')+"/neighbours"
                time.sleep(2)
                driver.get(user_link)
                user_names = driver.find_elements_by_class_name("user-list-name")
                for i in range(len(user_names)):
                    time.sleep(2)
                    user_link = user_names[i].find_element_by_tag_name('a').get_attribute('href')+"/neighbours"
                    time.sleep(2)
                    driver.get(user_link)
                    user_names = driver.find_elements_by_class_name("user-list-name")
                    for i in range(len(user_names)):
                        user.append(user_names[i].text)
                    driver.back()
                    time.sleep(2)
                    user_names = driver.find_elements_by_class_name("user-list-name")
                driver.back()
            except:
                continue
        user_names = driver.find_elements_by_class_name("user-list-name")
    user = list(dict.fromkeys(user))
    driver.quit()
    return user

In [4]:
load_df = pd.read_pickle("usersong")
load_df.drop_duplicates()

,count,user,song
0,120,Natidpp,Avril Lavigne-Tell Me It's Over
0,117,Natidpp,Louis Tomlinson-Habit
0,100,Natidpp,Louis Tomlinson-Defenceless
0,88,Natidpp,Taylor Swift-Paper Rings
0,84,Natidpp,Avril Lavigne-Dumb Blonde (feat. Nicki Minaj)
...,...,...,...
4480,28,KruccixGaheeKruccixGaheeKruccixGaheeKruccixGah...,XXXTentacion-Sad!
4480,27,KruccixGaheeKruccixGaheeKruccixGaheeKruccixGah...,Darren Styles-Stay Young - Original Mix
4480,27,KruccixGaheeKruccixGaheeKruccixGaheeKruccixGah...,Gucci Mane-Wake Up in the Sky
4480,27,KruccixGaheeKruccixGaheeKruccixGaheeKruccixGah...,Rich Brian-Dat $tick


In [1]:
#rzhan97_list = get_users("rzhan97")

In [7]:
with open('user.json') as f:
    user2 = json.load(f)

In [8]:
len(user2)

4481

In [11]:
rzhan97_small_list= sample(user2,2000)

In [12]:
#convert them into dataframe, also get the artist with the song, in case there are duplicate song name
song = []
artist = []
count = []
user = []
for i in range(len(rzhan97_small_list)):
    data = lookup_topsong(rzhan97_small_list[i])
    try:
        song.extend(data[0])
        count.extend(data[1])
        artist.extend(data[2])
        user.extend([rzhan97_small_list[i]]*50)
    except:
        continue


usersong_df = pd.DataFrame(zip(user,artist,song,count), columns =['user','artist','songs','count'])


In [13]:
usersong_df.head()

,user,artist,songs,count
0,sophiecatgirl,Conan Gray,People Watching,133
1,sophiecatgirl,Glass Animals,Heat Waves,92
2,sophiecatgirl,5 Seconds of Summer,Teeth,73
3,sophiecatgirl,The Weeknd,Take My Breath,67
4,sophiecatgirl,The Weeknd,High for This,62


In [14]:
usersong_df.tail()

,user,artist,songs,count
99995,FelipePandolfo,Marília Mendonça,Bebaça - Ao Vivo,69
99996,FelipePandolfo,The Weeknd,Blinding Lights,69
99997,FelipePandolfo,The Weeknd,Save Your Tears (with Ariana Grande) (Remix),69
99998,FelipePandolfo,Zé Neto & Cristiano,Notificação Preferida - Ao Vivo,69
99999,FelipePandolfo,Clarissa,nada contra (ciúme),68


In [15]:
#Change to another dataframe
usersong_rating = usersong_df

In [16]:
usersong_rating['song'] = usersong_rating["artist"] + "-" + usersong_rating["songs"]
usersong_rating = usersong_rating.drop(['artist'], axis=1)
usersong_rating = usersong_rating.drop(['songs'], axis=1)
usersong_rating.head(10)

,user,count,song
0,sophiecatgirl,133,Conan Gray-People Watching
1,sophiecatgirl,92,Glass Animals-Heat Waves
2,sophiecatgirl,73,5 Seconds of Summer-Teeth
3,sophiecatgirl,67,The Weeknd-Take My Breath
4,sophiecatgirl,62,The Weeknd-High for This
5,sophiecatgirl,59,Conan Gray-Astronomy
6,sophiecatgirl,55,Bella Thorne-Walk With Me
7,sophiecatgirl,55,Doja Cat-You Right
8,sophiecatgirl,55,The Neighbourhood-The Beach
9,sophiecatgirl,51,Gesaffelstein-Lost in the Fire (feat. The Weeknd)


In [17]:
usersong_rating.to_pickle("usersong")
load_df = pd.read_pickle("usersong")

In [20]:
load_df = load_df.drop_duplicates()
load_df.shape[0]

76764

In [23]:
load_df.head(10)

,user,count,song
0,sophiecatgirl,133,Conan Gray-People Watching
1,sophiecatgirl,92,Glass Animals-Heat Waves
2,sophiecatgirl,73,5 Seconds of Summer-Teeth
3,sophiecatgirl,67,The Weeknd-Take My Breath
4,sophiecatgirl,62,The Weeknd-High for This
5,sophiecatgirl,59,Conan Gray-Astronomy
6,sophiecatgirl,55,Bella Thorne-Walk With Me
7,sophiecatgirl,55,Doja Cat-You Right
8,sophiecatgirl,55,The Neighbourhood-The Beach
9,sophiecatgirl,51,Gesaffelstein-Lost in the Fire (feat. The Weeknd)


In [24]:
load_df.tail(10)

,user,count,song
99990,FelipePandolfo,72,Dua Lipa-Don't Start Now
99991,FelipePandolfo,72,Marília Mendonça-Apaixonadinha - Ao Vivo
99992,FelipePandolfo,70,Billie Eilish-all the good girls go to hell
99993,FelipePandolfo,70,Billie Eilish-everything i wanted
99994,FelipePandolfo,69,Billie Eilish-bellyache
99995,FelipePandolfo,69,Marília Mendonça-Bebaça - Ao Vivo
99996,FelipePandolfo,69,The Weeknd-Blinding Lights
99997,FelipePandolfo,69,The Weeknd-Save Your Tears (with Ariana Grande...
99998,FelipePandolfo,69,Zé Neto & Cristiano-Notificação Preferida - Ao...
99999,FelipePandolfo,68,Clarissa-nada contra (ciúme)


In [103]:
load_df[["count"]] = load_df[["count"]].apply(pd.to_numeric)

In [108]:
#load_df.iloc[[28250]]

In [109]:
#load_df.iloc[[28251]]

In [116]:
usersong_matrix_df = load_df.pivot_table(index='user', columns='song', values='count').fillna(0)

In [117]:
usersong_matrix_df.head(10)

song,$NOT-Moon & Stars (feat. Maggie Lindemann),"$uicideboy$-...And To Those I Love, Thanks For Sticking Around",$uicideboy$-KILL YOURSELF (Part III),$uicideboy$-Putrid Pride,$uicideboy$-Runnin' Thru the 7th with My Woadies,((( o )))-Nature's Joint,(G)I-DLE-$$$,(G)I-DLE-Blow Your Mind,(G)I-DLE-DON'T TEXT ME,(G)I-DLE-DUMDi DUMDi,...,"태양-눈,코,입(Eyes, Noes, Lips)",태양-웨딩드레스 (Wedding Dress),태연-가까이 (Closer),"택연, 우영, 수지, 김수현, JOO-드림하이",티아라-Lovey-Dovey,펜타곤 (PENTAGON)-Shine,하이라이트 (Highlight)-CELEBRATE,현빈-The Man,현아-Ice Cream (feat. Maboos),현아-빨개요
user,,,,,,,,,,,,,,,,,,,,,
-moonflower-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ANNTHNY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AbaobZen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Acxel21_,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Acxelll,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AdrianGkkgx,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AdrianoMc1989,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AidenOnlyy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Aidose,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [119]:
usersong_matrix_df.to_pickle("usersong_matrix_df")
usersong_matrix_load_df = pd.read_pickle("usersong_matrix_df")

In [120]:
#check the basic info, we have 1535 users and 24814 songs
usersong_matrix_load_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1535 entries, -moonflower- to zyranie
Columns: 24814 entries, $NOT-Moon & Stars (feat. Maggie Lindemann) to 현아-빨개요
dtypes: float64(24814)
memory usage: 290.6+ MB


# Get users' info

In [129]:
#Get user's info
#convert them into dataframe,also get user's infoun
name = []
country = []
playcount = []
for i in range(len(rzhan97_small_list)):
    data = lookup_userinfo(rzhan97_small_list[i])
    #use this function to get the 
    try:
        name.append(data[0])
        country.append(data[1])
        playcount.append(data[2])
    except:
        continue


user_df = pd.DataFrame(zip(rzhan97_small_list,name,country,playcount), columns =['user','name','country','playcount'])
user_df.to_pickle("user_df")

In [133]:
user_df.tail()

,user,name,country,playcount
1995,kayovicc,,None,7775
1996,ltybill,,None,2813
1997,sevenone0,codephoenix,Canada,126445
1998,kingofpale,Jose Torres,None,59596
1999,FelipePandolfo,,None,57289


# Get songs' info

In [137]:
#Prepare for the song's metadata 
songs_df = usersong_df[['songs', 'artist']] 
songs_df = songs_df.drop_duplicates()
songs_list = songs_df.songs.tolist()
artists_list = songs_df.artist.tolist()
print(len(songs_list))

24814


### Question: Do we have another quicker way to do this?

In [ ]:
#Get each song's info for later use
duration = []
listeners = []
playcount = []
album = []
tags = []
published_date = []

for i in range(len(songs_list)):
    data = lookup_songinfo(songs[i],artists_list[i])
    #if any records get error
    try:
        duration.append(data[0])
        listeners.append(data[1])
        playcount.append(data[2])
        album.append(data[3])
        tags.append(data[4])
        published_date.append(data[5])
    except:
        continue


user_df = pd.DataFrame(zip(songs,artists_list,duration,listeners,playcount,album,tags,published_date), columns =['song','artist','duration','listeners','playcount','album','tags','published_date'])


# Use matrix to fit into collaborative filtering package surprise

In [139]:
#!source activate py36;pip install scikit-surprise

In [145]:
import pandas as pd
from surprise import Dataset
from surprise import Reader
from surprise import KNNWithMeans
from surprise import Dataset
from surprise.model_selection import GridSearchCV

In [142]:
data = Dataset.load_builtin("ml-100k")
print(data)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /Users/zoe/.surprise_data/ml-100k


In [143]:
print(data)

In [147]:
sim_options = {
    "name": ["msd", "cosine"],
    "min_support": [3, 4, 5],
    "user_based": [False, True],
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

In [ ]:
reader = Reader(rating_scale=(1, 5))
surprise_data = Dataset.load_from_df(df[[user_col, item_col, rating_col]], reader)

In [ ]:
algo = SVD()